In [1]:
import pandas as pd
        

In [5]:
class Purprd_Process:
    def __init__(self, purprd_url, prod_cat_url):
        self.purprd = pd.read_csv(purprd_url, encoding='CP949')
        self.prod_cat = pd.read_csv(prod_cat_url, encoding='CP949')
        
    def period_cat(self, x):
        if x <= 6: return 1
        else : return 2
        
    def week_cat(self, x):
        if x in [5,6]: return '주말'
        else : return '주중'
        
    def time_cat(self, x):
        if x in [1,2,3,4,5,6]: return '새벽'
        elif x in [7,8,9,10,11,12]: return '오전'
        elif x in [13,14,15,16,17,18]: return '오후'
        else: return '밤'
        
    def yoil_cat(self, x):
        if x == 0: return '월'
        elif x == 1: return '화'
        elif x == 2: return '수'
        elif x == 3: return '목'
        elif x == 4: return '금'
        elif x == 5: return '토'
        else: return '일'
        
        
    def merge_df(self):
        self.df = pd.merge(self.purprd, self.prod_cat[['소분류코드','분류 체계','중분류명','소분류명']], on = '소분류코드')
        del self.purprd
        del self.prod_cat
        self.df = self.df.rename(columns = {'제휴사':'channel', '영수증번호':'receipt_no', 
                         '대분류코드':'code_dae', '중분류코드':'code_jung', '소분류코드':'code_so',
                         '고객번호':'cust_no', '점포코드':'branch_no', 
                         '구매일자':'pur_date', '구매시간':'pur_hour', '구매금액':'pur_amount',
                         '분류 체계':'cat_dae', '중분류명':'cat_jung', '소분류명':'cat_so'})
        
    
    def categorical(self):
        self.df['y_month'] = self.df['pur_date'].astype(str).apply(lambda x : x[:-3].replace('-', '_'))
        print ('1')
        self.df['pur_date'] = pd.to_datetime(self.df['pur_date'].astype(str))
        self.df['year'] = self.df['pur_date'].dt.year
        self.df['month'] = self.df['pur_date'].dt.month
        
        self.df['yoil'] = self.df['pur_date'].dt.dayofweek
        self.df['yoil'] = self.df['yoil'].apply(self.yoil_cat)
        print (2)
        self.df['half'] = self.df['month'].apply(self.period_cat)
        self.df['week'] = self.df['yoil'].apply(self.week_cat)
        self.df['pur_time'] = self.df['pur_hour'].apply(self.time_cat)
        print (3)
        self.df['period'] = self.df['year'].astype(str)+'_'+ self.df['half'].astype(str)
        self.df['pur_week_time'] = self.df['week'] +'_'+self.df['pur_time']
        self.df['quarter'] = self.df['pur_date'].dt.quarter
        self.df['y_quarter'] = self.df['year'].astype(str)+'_'+ self.df['quarter'].astype(str)        
        
    def total_processing(self, save_url=None):
        self.merge_df()
        self.categorical()
        if url:
            self.df.to_csv(save_url, index=False)
            print ('save Complete!')
        
        return self.df

In [ ]:
func = Purprd_Process('원본dataframe/구매상품TR.txt','원본dataframe/분류체계_0927.csv')

df = func.total_processing('data.csv')
# df.head()